In [1]:
import numpy as np
import pandas as pd
import re
from BondGraphTools import *
from network_modelling import *
from network_visualization import *

import ndlib.models.epidemics.SISModel as sis

from bokeh.io import show, output_notebook

from os import environ
output_notebook()

Loading BokehJS ...

In [2]:
user = environ["USERNAME"]
#from work
datafolder = 'C:/Users/DavideMariani/Tradeteq Dropbox/Davide Mariani/thesis_project/'

#from home
#datafolder = "C:/Users/{}/Dropbox/University/MscDataScience_Birkbeck/thesis_project/data/".format(user)
inst = pd.read_pickle(datafolder+'snapshots_cd-d-cstats.pkl')

In [3]:
a = inst.loc[inst['is_due'], 'purchase_amount'] - inst.loc[inst['is_due'], 'total_repayment']
sum(a<0.009)

49534

In [4]:
inst.columns[:54]

Index(['customer_id', 'customer_name_1', 'debtor_id', 'debtor_name_1',
       'invoice_number', 'invoice_date', 'due_date', 'invoice_amount',
       'purchase_amount', 'purchase_amount_open', 'discharge_type',
       'discharge_amount', 'discharge_date', 'posting_date',
       'transaction_type', 'document_stack_id', 'booking_text', 'input_date',
       'creation_date', 'factoring_type', 'debt_collection_date',
       'last_payment_date', 'reminder_date', 'test_feature_tested',
       'test_characteristic_id', 'bill_id', 'cancellation_date', 'value_date',
       'currency', 'purchase_examination', 'prosecution', 'deduction_amount',
       'payment_amount', 'payment_date', 'document_status', 'uid', 'ttype',
       'nrecords', 'ttypeset', 'discharge_loss', 'has_impairment1',
       'has_impairment2', 'has_impairment3', 'is_pastdue90', 'is_pastdue180',
       'has_prosecution', 'last_payment_amount', 'total_repayment',
       'total_impairment', 'is_open', 'we_payment_share', 'has_purchas

In [5]:
inst.head()

,customer_id,customer_name_1,debtor_id,debtor_name_1,invoice_number,invoice_date,due_date,invoice_amount,purchase_amount,purchase_amount_open,...,sshot_61_c_repaid_c,sshot_61_c_pastdue90_c,sshot_61_c_pastdue180_c,sshot_61_c_trend_a,sshot_61_c_we_payment_share,sshot_61_c_pd_mismatch_mean,sshot_61_c_pd_mismatch_std,sshot_61_c_repaid_r,sshot_61_c_pastdue90_r,sshot_61_c_pastdue180_r
uid,,,,,,,,,,,,,,,,,,,,,
2744:79/231,2004008,jobs united GmbH,79,Quadroni Linard,2744,2013-07-23,2013-08-02,913.70,0.0,0.0,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2861:79/232,2004008,jobs united GmbH,79,Quadroni Linard,2861,2013-07-30,2013-08-09,2233.45,0.0,0.0,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,0.0,0.0
2932:79/233,2004008,jobs united GmbH,79,Quadroni Linard,2932,2013-08-06,2013-08-16,1370.50,0.0,0.0,...,0.0,0.0,0.0,7.185198,NaN,NaN,NaN,0.0,0.0,0.0
1472:489/688,2004009,PM Personal GmbH,489,Style Interiors,1472,2013-08-13,2013-08-23,9195.10,0.0,0.0,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2042:512/645,2004009,PM Personal GmbH,512,Elektropartner AG,2042,2013-08-13,2013-08-23,4594.60,0.0,0.0,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,0.0,0.0


In [6]:
#selecting one snapshot
snapshot = 'sshot_61_'
s61 = inst[inst[snapshot]][[c for c in inst.columns if snapshot in c] \
                      +['has_purchase', 'due_date', 'invoice_date', 'customer_name_1', 
                       'debtor_name_1', 'customer_id', 'debtor_id', 'uid']]
s61 = s61[s61.has_purchase]
#s35.transpose()

In [8]:
s61.shape

(46353, 53)

In [9]:
#max number of pastdues instruments between a customer and a debtor at snapshot 35
s61.groupby(['customer_name_1', 'debtor_name_1']).sshot_61_cd_pastdue90_c.max()

customer_name_1                                  debtor_name_1                                
ABC Steel Products GBP                           Amerex Ltd.                                       4.0
                                                 Atlantic Steel Processing Ltd                     0.0
                                                 BRANDVIK LIMITED                                  0.0
                                                 C&S NEATH STEEL SERVICES LTD.                     0.0
                                                 CJ Upton & Sons Ltd                               3.0
                                                 Chainbridge Steel (North East) Ltd                0.0
                                                 Cutler & Woolf Steel Ltd                          4.0
                                                 D.N.T Company Ltd                                 0.0
                                                 Davro Steel Ltd                 

In [10]:
s61.columns

Index(['sshot_61_', 'sshot_61_payment_date', 'sshot_61_payment_amount',
       'sshot_61_last_payment_amount', 'sshot_61_last_payment_date',
       'sshot_61_total_repayment', 'sshot_61_is_pastdue90',
       'sshot_61_is_pastdue180', 'sshot_61_payment_date_mismatch',
       'sshot_61_is_open', 'sshot_61_we_payment_share', 'sshot_61_is_due',
       'sshot_61_cd_lent_c', 'sshot_61_cd_repaid_c', 'sshot_61_cd_pastdue90_c',
       'sshot_61_cd_pastdue180_c', 'sshot_61_cd_trend_a',
       'sshot_61_cd_we_payment_share', 'sshot_61_cd_pd_mismatch_mean',
       'sshot_61_cd_pd_mismatch_std', 'sshot_61_cd_repaid_r',
       'sshot_61_cd_pastdue90_r', 'sshot_61_cd_pastdue180_r',
       'sshot_61_d_lent_c', 'sshot_61_d_repaid_c', 'sshot_61_d_pastdue90_c',
       'sshot_61_d_pastdue180_c', 'sshot_61_d_trend_a',
       'sshot_61_d_we_payment_share', 'sshot_61_d_pd_mismatch_mean',
       'sshot_61_d_pd_mismatch_std', 'sshot_61_d_repaid_r',
       'sshot_61_d_pastdue90_r', 'sshot_61_d_pastdue180_r',
  

In [11]:
fields = ['sshot_61_cd_lent_c', 'sshot_61_cd_repaid_c', 'sshot_61_cd_pastdue90_c',
       'sshot_61_cd_pastdue180_c', 'sshot_61_cd_trend_a',
       'sshot_61_cd_we_payment_share', 'sshot_61_cd_pd_mismatch_mean',
       'sshot_61_cd_pd_mismatch_std', 'sshot_61_cd_repaid_r',
       'sshot_61_cd_pastdue90_r', 'sshot_61_cd_pastdue180_r',
       'sshot_61_d_lent_c', 'sshot_61_d_repaid_c', 'sshot_61_d_pastdue90_c',
       'sshot_61_d_pastdue180_c', 'sshot_61_d_trend_a',
       'sshot_61_d_we_payment_share', 'sshot_61_d_pd_mismatch_mean',
       'sshot_61_d_pd_mismatch_std', 'sshot_61_d_repaid_r',
       'sshot_61_d_pastdue90_r', 'sshot_61_d_pastdue180_r',
       'sshot_61_c_lent_c', 'sshot_61_c_repaid_c', 'sshot_61_c_pastdue90_c',
       'sshot_61_c_pastdue180_c', 'sshot_61_c_trend_a',
       'sshot_61_c_we_payment_share', 'sshot_61_c_pd_mismatch_mean',
       'sshot_61_c_pd_mismatch_std', 'sshot_61_c_repaid_r',
       'sshot_61_c_pastdue90_r', 'sshot_61_c_pastdue180_r']

In [13]:
s61.sshot_61_d_pd_mismatch_std.max()

315.9588424130743

In [14]:
dtemp = prepare_connections(s61, fields = fields)

In [30]:
dtemp.sshot_35_d_pd_mismatch_std.max()

60.33517492585344

In [15]:
dtemp

,customer_id,uid,customer_name_1,debtor_id,debtor_name_1,sshot_61_cd_lent_c,sshot_61_cd_repaid_c,sshot_61_cd_pastdue90_c,sshot_61_cd_pastdue180_c,sshot_61_cd_trend_a,...,sshot_61_c_repaid_c_2,sshot_61_c_pastdue90_c_2,sshot_61_c_pastdue180_c_2,sshot_61_c_trend_a_2,sshot_61_c_we_payment_share_2,sshot_61_c_pd_mismatch_mean_2,sshot_61_c_pd_mismatch_std_2,sshot_61_c_repaid_r_2,sshot_61_c_pastdue90_r_2,sshot_61_c_pastdue180_r_2
24,2004016,1112:717/1157,inter personal GmbH,717,Team Fortis GmbH,4.0,0.0,0.0,0.0,-6.717768,...,0.0,0.0,0.0,-5.394082,NaN,NaN,NaN,0.000000,0.000000,0.000000
28,2004004,600188:1121/172,CDP SWISS AG,1121,Dourado Cabral Sensation Daniella Caroline Dou...,1.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
32,2004008,4325GU:244/228,jobs united GmbH,244,AZ Elektro AG,1.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.439237,NaN,NaN,NaN,0.000000,0.000000,0.000000
36,2004004,600189:1090/174,CDP SWISS AG,1090,Natural Skin GmbH,1.0,0.0,1.0,0.0,0.000000,...,0.0,3.0,2.0,-7.580725,NaN,NaN,NaN,0.000000,1.000000,0.666667
765,che2004001,NM-618-14:1020000/57,Speo Sàrl,1020000,Bertrandt SA,26.0,0.0,0.0,0.0,3.744508,...,0.0,0.0,0.0,-4.231310,NaN,NaN,NaN,0.000000,0.000000,0.000000
829,2004009,3482:699/706,PM Personal GmbH,699,Stöckli Bodenbeläge AG,7.0,0.0,0.0,0.0,-4.869524,...,0.0,0.0,0.0,-2.458218,NaN,NaN,NaN,0.000000,0.000000,0.000000
833,2004014,1123:692/1076,ONJO AG,692,Leu und Partner Elektro AG,1.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
852,che2004011,14-101:1070029/118,Swiscovital AG,1070029,GEHE Pharma Handel GmbH,18.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
1470,2004005,15003:3504/196,PAMAG Personal St. Gallen GmbH,3504,ZKD Büromöbel AG,12.0,0.0,0.0,0.0,6.131416,...,0.0,0.0,0.0,6.131416,NaN,NaN,NaN,0.000000,0.000000,0.000000
1506,2004011,15-179:1040009/837,Creative Promotions GmbH (CHF02),1040009,Diageo Singapore Pte. Ltd.,5.0,0.0,2.0,0.0,-8.582284,...,0.0,2.0,0.0,-8.582284,NaN,NaN,NaN,0.000000,0.400000,0.000000


In [16]:
nodes = create_nodes_df(dtemp)

In [17]:
nodes.head()

,Company_Name,ID,Type,Type_2
0,inter personal GmbH,'2004016',seller,seller
1,CDP SWISS AG,'2004004',seller,seller
2,jobs united GmbH,'2004008',seller,seller
3,Speo Sàrl,'che2004001',seller,seller
4,PM Personal GmbH,'2004009',seller,seller


In [18]:
s61.columns

Index(['sshot_61_', 'sshot_61_payment_date', 'sshot_61_payment_amount',
       'sshot_61_last_payment_amount', 'sshot_61_last_payment_date',
       'sshot_61_total_repayment', 'sshot_61_is_pastdue90',
       'sshot_61_is_pastdue180', 'sshot_61_payment_date_mismatch',
       'sshot_61_is_open', 'sshot_61_we_payment_share', 'sshot_61_is_due',
       'sshot_61_cd_lent_c', 'sshot_61_cd_repaid_c', 'sshot_61_cd_pastdue90_c',
       'sshot_61_cd_pastdue180_c', 'sshot_61_cd_trend_a',
       'sshot_61_cd_we_payment_share', 'sshot_61_cd_pd_mismatch_mean',
       'sshot_61_cd_pd_mismatch_std', 'sshot_61_cd_repaid_r',
       'sshot_61_cd_pastdue90_r', 'sshot_61_cd_pastdue180_r',
       'sshot_61_d_lent_c', 'sshot_61_d_repaid_c', 'sshot_61_d_pastdue90_c',
       'sshot_61_d_pastdue180_c', 'sshot_61_d_trend_a',
       'sshot_61_d_we_payment_share', 'sshot_61_d_pd_mismatch_mean',
       'sshot_61_d_pd_mismatch_std', 'sshot_61_d_repaid_r',
       'sshot_61_d_pastdue90_r', 'sshot_61_d_pastdue180_r',
  

In [19]:
edges = create_edges_df(dtemp, fields = ['sshot_61_cd_pastdue90_c'])

In [22]:
edges[edges.sshot_61_cd_pastdue90_c==4]

,xs,ys,edges_couples,sshot_61_cd_pastdue90_c
Meier Schaltanlagen,Meier Schaltanlagen,Corrodi + Keller AG,"(Meier Schaltanlagen, Corrodi + Keller AG)",4.0
Lemania Interim SA,Lemania Interim SA,Eddy Fritz,"(Lemania Interim SA, Eddy Fritz)",4.0
ABC Steel Products GBP,ABC Steel Products GBP,South Essex Stockholders Ltd,"(ABC Steel Products GBP, South Essex Stockhold...",4.0
ABC Steel Products GBP,ABC Steel Products GBP,Cutler & Woolf Steel Ltd,"(ABC Steel Products GBP, Cutler & Woolf Steel ...",4.0
ABC Steel Products GBP,ABC Steel Products GBP,Amerex Ltd.,"(ABC Steel Products GBP, Amerex Ltd.)",4.0
P+P Personal AG,P+P Personal AG,Carlo Massaro AG,"(P+P Personal AG, Carlo Massaro AG)",4.0
United Personal Management AG,United Personal Management AG,"Baumann, Koelliker AG","(United Personal Management AG, Baumann, Koell...",4.0
Meier Schaltanlagen,Meier Schaltanlagen,EW Elektro Widmer AG,"(Meier Schaltanlagen, EW Elektro Widmer AG)",4.0
Lemania Interim SA,Lemania Interim SA,FASTRENO Sàrl,"(Lemania Interim SA, FASTRENO Sàrl)",4.0
Lemania Interim SA,Lemania Interim SA,CTH CONSTRUCTION ET TRAVAUX,"(Lemania Interim SA, CTH CONSTRUCTION ET TRAVAUX)",4.0


In [23]:
n, edges, nodes = create_network(edges, nodes)

C:\Users\DavideMariani\Tradeteq Dropbox\Davide Mariani\thesis_project\networkAnalysisForML\network_modelling.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  nodes['centrality'] = [centrality[n] for n in list(nodes['Company_Name'])]
C:\Users\DavideMariani\Tradeteq Dropbox\Davide Mariani\thesis_project\networkAnalysisForML\network_modelling.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  nodes['size'] = np.around(np.interp(nodes['centrality'], (nodes['centrality'].min(), nodes['centrality'].max()), (min_size, max_size)),2)


In [24]:
nodes.head()

,Company_Name,ID,Type,Type_2,centrality,size
0,inter personal GmbH,'2004016',seller,seller,0.000383,6.00
115,Team Fortis GmbH,'717',buyer,buyer,0.000383,6.00
1,CDP SWISS AG,'2004004',seller,seller,0.000766,6.03
116,Dourado Cabral Sensation Daniella Caroline Dou...,'1121',buyer,buyer,0.000383,6.00
2,jobs united GmbH,'2004008',seller,seller,0.000383,6.00


In [29]:
g = visualize_graph(n, edges, nodes, nx_k=0.02)

In [30]:
show(g)

In [31]:
import networkx as nx

In [32]:
def node_neighborhood(G, node, n, nfilter=[]):
    neigh = nx.single_source_dijkstra(G, node)
    path_lengths = neigh[0]
    paths = neigh[1]
    print(path_lengths)
    all_neigh = [node for node, length in path_lengths.items() if length <= n]
    
    if len(nfilter)==0:
        return all_neigh
    else:
        nodes = set()
        for f in nfilter:
            for g in all_neigh:
                if f in set(paths[g]):
                    nodes.add(g)
        return list(nodes)
            

#node_neighborhood(n, 'BluePlan AG',4, ['Carlo Massaro AG'])

In [43]:
def create_graph(graph, edges, nodes, nodes_name_column='Company_Name', nodes_size_column='size', nodes_type_column='Type_2',
                    title = 'Network Graph', plot_w = 900, plot_h = 900, file_output = '', nx_k=0.028, nx_iterations=25,
                      nodes_colors = [TTQcolor['sky'], TTQcolor['Salmon'], TTQcolor['marketplaceOrange']], to_highlight = '',
                      edges_colors = [TTQcolor['whiteGrey'], TTQcolor['warningRed']]):

    """
    This function will give visual attributes to the graph.
    """

    graph = from_networkx(graph, nx.spring_layout, k=nx_k, iterations=nx_iterations)

    #unfortunately the use of list comprehension at next step is necessary
    #since bokeh doesn't seem to support other collections like Series or arrays
    graph.node_renderer.data_source.data['index'] = [i for i in nodes['Company_Name']] #setting the company names
    graph.node_renderer.data_source.data['size'] = [s for s in nodes['size']] #setting node sizes
    graph.node_renderer.data_source.data['type'] = [t for t in nodes['Type_2']] #setting types

    graph.node_renderer.glyph = Circle(size='size', fill_color=factor_cmap('type', nodes_colors,    #creating nodes
                                                                    ['buyer', 'seller and buyer', 'seller']),
                                                                     fill_alpha=0.8, line_color='white', line_width=0.5)

    graph.node_renderer.nonselection_glyph = Circle(size='size', fill_color=factor_cmap('type', nodes_colors, #creating non-selected nodes
                                                                nodes['Type_2'].unique()),
                                               fill_alpha=0.1, line_alpha=0.05)

    
    if len(to_highlight)>0:
        graph.edge_renderer.data_source.data['highlight']=list(edges[to_highlight])
        graph.edge_renderer.glyph = MultiLine(line_color=linear_cmap('highlight', edges_colors, 
                                                                     False,True), line_width=0.15, line_alpha=0.85)
    else:
        graph.edge_renderer.glyph = MultiLine(line_width=0.15, line_alpha=0.85)

    graph.node_renderer.hover_glyph = Circle(size='size', fill_alpha=0.0, line_width=3, line_color='green') #creating hover settings for circles
    graph.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=0.8) #creating hover settings for edges

    graph.selection_policy = NodesAndLinkedEdges()
    graph.inspection_policy = NodesAndLinkedEdges()

    return graph

In [23]:
#f = create_graph(n, edges, nodes, nx_k=0.08)

In [36]:
#isolation of subgraph - with their length
subnetworks = [c for c in sorted(nx.connected_components(n), key=len, reverse=True)]
len(subnetworks)

49

In [38]:
subnetworks_size = [len(c) for c in subnetworks]
subnetworks_size

[1953,
 145,
 79,
 45,
 37,
 23,
 22,
 21,
 20,
 17,
 17,
 14,
 13,
 13,
 12,
 12,
 11,
 11,
 10,
 9,
 9,
 8,
 8,
 7,
 7,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [39]:
set(subnetwork_sizes)

{2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 17,
 20,
 21,
 22,
 23,
 37,
 45,
 79,
 145,
 1953}

In [79]:
snodes = subnetworks[11]
s = n.subgraph(snodes)

In [80]:
edges_s = edges[edges['xs'].isin(snodes) & edges['ys'].isin(snodes)]
nodes_s = nodes[nodes['Company_Name'].isin(snodes)]

vs = visualize_graph(s, edges_s, nodes_s, nx_k=0.02, to_highlight='sshot_61_cd_pastdue90_c')
show(vs)